# SUBJECT ,VERB AND OBJECT EXTRACTION_POS

In [11]:
%%time
import nltk
import numpy as np
import pandas as pd # USE OF PANDAS FOR MANUPILATION AND CONSTRUCTION OF DATAFRAME
import spacy
from spacy.gold import GoldParse # USE OF SPACY 
nlp = spacy.load('en_core_web_sm')

Wall time: 2.41 s


In [12]:
%cd "D:\Ria\Internship\Internshala\Dowell"

D:\Ria\Internship\Internshala\Dowell


In [13]:
data=pd.read_excel("SEARCH_STRING_GOOGLE_AUTOCOMPLETE.xlsx")

In [14]:
# function to extract subject ,verb and object
def subject_verb_object_pos(x):

    doc=nlp(x)
    clausal_subject = ""
    nominal_subject = ""
    indirect_object = ""
    Object          = ""
    preposition_Object=""
    verb=""
    subject=""
    
    for word  in doc:
       
        
        if word.dep_ in ["nsubj"]:
            subject=word.text
 
        elif word.dep_== "csubj":
                clausal_subject=word.text
        elif word.dep_  == "iobj":
                indirect_object=word.text
        elif word.dep_  == "dobj":
                Object=word.text
        elif word.dep_ == "pobj":
                preposition_Object=word.text
        elif word.pos_=='VERB':
                verb=word.text
    return {'subject':subject,'clausal_subject':clausal_subject,'verb':verb,'indirect_object':indirect_object,'direct_object':Object,'p_object':preposition_Object}
     
                
        
 

In [43]:
def pos(x):
    a=[]
    wordList=nltk.word_tokenize(x)
    doc=nlp(' '.join(wordList))
    for token  in doc:
        a.append({'text':token.text,'pos_':token.pos_})
    
    
#     pos_tagging=[]
#     for i in pos:
#         pos_tagging.append(i)
#     pos_tag=pd.DataFrame(pos_tagging)
    return a

In [5]:
subject_verb_object_pos('live score ipl 2020')

{'subject': '',
 'clausal_subject': '',
 'verb': '',
 'indirect_object': '',
 'direct_object': '',
 'p_object': ''}

In [15]:
%%time
pos_tag=[]
search_string=[]
for i in data["SEARCH STRING "].astype('str'):
        pos_tag.append(subject_verb_object_pos(i))
        search_string.append(i)

Wall time: 7.1 s


In [16]:
searchstring=pd.DataFrame(search_string)
pos_tag=pd.DataFrame(pos_tag)

In [17]:
d=pd.concat([searchstring,pos_tag],axis=1)

In [54]:
#convert NAN values to empty string
nan_value=float("NAN")
d.replace("",nan_value,inplace=True)
d.dropna(subset=['subject', 'clausal_subject',
                  'verb', 'indirect_object',   'direct_object',
              'p_object'],how='all',inplace=True)
d=d.fillna('')
d.rename(columns={0:'searchstring'},inplace=True)

In [44]:
%%time
pos_tag=[]
search_string=[]
for i in data["SEARCH STRING "].astype('str'):
        pos_tag.append(pos(i))
        search_string.append(i)
data_=pd.DataFrame(search_string,np.array(pos_tag).flatten(),columns=['searchstring']).reset_index().rename(columns={'index':'summary'}) 
data_=data_[['searchstring','summary']]  
data_['summary']=data_['summary'].apply(lambda x:str(x).replace('(','').replace('[','').replace(')','').replace(']','').replace("'",'').replace('{','').replace('}','').replace('text:','').replace('pos_:','').replace('tag_','').replace('dep_:','').replace(':',''))
data_=data_[['searchstring','summary']]  
data_m=data_['summary'].astype('str').str.split(",",expand=True)
final=pd.concat([data_['searchstring'],data_m],axis=1)
final.columns=['searchstring']+['word','pos']*(len(final.columns[1:])//2)
final=final.fillna(' ')

Wall time: 2.35 s


In [65]:
pd.merge(d,final,on='searchstring',how='right').sort_values(by='searchstring').fillna('')


,searchstring,subject,clausal_subject,verb,indirect_object,direct_object,p_object,word,pos,word,pos,word,pos,word,pos,word,pos,word,pos
348,2002-10-01 00:00:00,,,,,,,2002,NUM,-,SYM,10,NUM,-,SYM,01,NUM,000000,NUM
111,aadhar card,,,,,,,aadhar,NOUN,card,NOUN,,,,,,,,
112,aaj tak,,,,,,,aaj,INTJ,tak,PROPN,,,,,,,,
0,aaj tak live,tak,,live,,,,aaj,INTJ,tak,PROPN,live,VERB,,,,,,
113,abdul samad,,,,,,,abdul,PROPN,samad,PROPN,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,zomato,,,,,,,zomato,PROPN,,,,,,,,,,
511,zoom,,,,,,,zoom,PROPN,,,,,,,,,,
512,zoom app,,,,,,,zoom,PROPN,app,PROPN,,,,,,,,
110,zoom download,,,zoom,,,,zoom,VERB,download,PROPN,,,,,,,,


,searchstring,subject,clausal_subject,verb,indirect_object,direct_object,p_object
2,aaj tak live,tak,,live,,,
4,aditi hundia,,,,,hundia,
23,bank of baroda,,,,,,baroda
26,bayern vs sevilla,,,,,,sevilla
41,birthday wishes,birthday,,wishes,,,
...,...,...,...,...,...,...,...
465,who invented telescope,who,,invented,,telescope,
466,word to pdf,,,,,,pdf
497,youtube to mp3,,,,,,mp3
502,zee news live,news,,,,,
